In [1]:
bval52 = [5.5845,5.5840,5.5835,5.5830,5.5825,5.5820,5.5815,5.5810,5.5805,5.5800,5.5795,5.5790,5.5785,5.5780,5.5775]
bval73 = [5.7485,5.7480,5.7475,5.7470,5.7465,5.7460,5.7455,5.7450,5.7445,5.7440,5.7435,5.7430,5.7425,5.7420,5.7415,5.7410,5.7405]
bval2011 = [5.536,5.5355,5.535,5.5345,5.534,5.5335,5.533,5.5325,5.532,5.5315,5.531,5.5305,5.53,5.5295,5.529,5.5285,5.528,5.5275,5.527,5.5265,5.526,5.5255]


In [2]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import CompleteHist as ch
import pylab
from matplotlib.gridspec import GridSpec
%matplotlib 

Using matplotlib backend: TkAgg


In [15]:
se52 = np.load('52se.npy')
se2011 = np.load('2011se.npy')
ch.parsePrune('soft')
ch.parseData()

In [4]:
import SG
from scipy import interpolate

In [5]:
def search(data, t):
    seq = data[0]    
    min = 0
    max = len(seq) - 1
    while True:        
        m = (min + max) // 2
        if max < min:
            if np.abs(t-seq[m]) < np.abs(t-seq[m-1]):          
                return m
            else:
                return m-1        
        if seq[m] < t:
            min = m + 1
        elif seq[m] > t:
            max = m - 1
        else:
            return m
            

def clip(d,min,max):
    data = np.array(d)
    li = search(data,min)
    ri = search(data,max)
    return data[:,li:ri+1]
    

In [6]:
def normalize(data):
    minv = min(data)
    maxv = max(data)
    hold = np.array(data)
    for i in range(len(hold)):
        hold[i] = (hold[i]-minv)/(maxv-minv)
    return hold

In [10]:
bfieldval = bval52
n = 1024
smoothval = 51
poly = 1
interpdim = 2 
data = ch.getData()
cmlabel = 'Normalized $\Delta$R$_D$ (k$\Omega$)'
data = [x for (y,x) in sorted(zip(bfieldval,data), key=lambda pair: pair[0])]
bval = sorted(bfieldval)    
xmin = None
xmax = 0
for i in data:
    if i[0][0] > xmin:
        xmin = i[0][0]
    if i[0][-1] < xmax:
        xmax = i[0][-1]
xmin += 0.5
xmax -= 0.5
xv = np.linspace(xmin,xmax,n)
intv = list()
for i in data:
    #Try flipping filtering and interp
    dataclip = clip(i,xmin,xmax)
    dataclip[1] = SG.savitzky_golay(dataclip[1],smoothval,poly)
    tck = interpolate.splrep(dataclip[0], dataclip[1])
    y = interpolate.splev(xv,tck)
    intv.append(normalize(y))
xb = np.linspace(bval[0],bval[-1],n)
b = list()
for i in range(len(intv[0])):
    hold = list()
    for j in range(len(intv)):
        hold.append(intv[j][i])
    b.append(hold)
intb = list()
for i in b:
    tck = interpolate.splrep(bval,i,k=interpdim)
    y = interpolate.splev(xb,tck)
    intb.append(y)
Z = np.array(intb)
X,Y = pylab.meshgrid(xb,xv)



In [11]:
colortype = ch.yellowredblue
f, ax = plt.subplots()
plt.pcolormesh(X,Y,Z,vmin=0,vmax=1,cmap=colortype)
plt.ylim(xmin,xmax)
plt.xlim(bval[0],bval[-1])
plt.ylabel('Gate Voltage (mV)')
plt.xlabel('B Field (T)')
cbar = plt.colorbar()
cbar.set_label(cmlabel)
ax.get_xaxis().get_major_formatter().set_useOffset(False)

plt.show()

In [12]:
def percentile_normalize(data,p=1):
    minv = np.percentile(data,p)
    maxv = np.percentile(data,100-p)
    hold = np.array(data)
    for i in range(len(hold)):
        hold[i] = (hold[i]-minv)/(maxv-minv)
    return hold

In [16]:
bfieldval = bval52
n = 1024
smoothval = 51
poly = 2
interpdim = 2
data = ch.getPrune()
cmlabel = 'R$_D$ (k$\Omega$)'
data = [x for (y,x) in sorted(zip(bfieldval,data), key=lambda pair: pair[0])]
bval = sorted(bfieldval)    
xmin = -100
xmax = 0
for i in data:
    if i[0][0] > xmin:
        xmin = i[0][0]
    if i[0][-1] < xmax:
        xmax = i[0][-1]
xmin += 0.5
xmax -= 0.5
xv = np.linspace(xmin,xmax,n)
intv = list()
for i in data:
    #Try flipping filtering and interp
    dataclip = clip(i,xmin,xmax)
    dataclip[1] = SG.savitzky_golay(dataclip[1],smoothval,poly)
    tck = interpolate.splrep(dataclip[0], dataclip[1],k=1)
    y = interpolate.splev(xv,tck)
    intv.append(percentile_normalize(y,1))
xb = np.linspace(bval[0],bval[-1],n)
b = list()
for i in range(len(intv[0])):
    hold = list()
    for j in range(len(intv)):
        hold.append(intv[j][i])
    b.append(hold)
intb = list()
for i in b:
    tck = interpolate.splrep(bval,i,k=interpdim)
    y = interpolate.splev(xb,tck)
    intb.append(y)
Z = np.array(intb)
X,Y = pylab.meshgrid(xb,xv)



In [18]:
gs = GridSpec(1000,1000)


In [39]:
#Has Vertical Scale
xin = 3.1
yin = 3.1
img = xin*0.9
y_len = int(img/yin*1000)
colortype = ch.yellowredblue
f= plt.figure(figsize=[3.1,3.3])
ax = f.add_subplot(gs[100:y_len,100:])
im = ax.pcolormesh(X,Y,Z,vmin=0,vmax=1,cmap=colortype)
ax.get_xaxis().get_major_formatter().set_useOffset(False)
plt.xticks([5.578,5.581,5.584])
plt.yticks([-30,-20,-10])
ax.set_ylim(xmin,xmax)
ax.set_xlim(bval[0],bval[-1])
plt.ylabel('V$_{p}$ (mV)',labelpad=-1)

plt.xlabel('B (T)')
cax = f.add_subplot(gs[65:90,400:700])

cbar = plt.colorbar(im,cax=cax,ticks=[0.0,1.0],orientation='horizontal')
cax.xaxis.set_label_position('top')
cax.set_xlabel('R$_D$ (k$\Omega$)')
cax.set_xticklabels([10.8,11.5])
cax.xaxis.labelpad = 2
cax.tick_params(axis='x',which='both',bottom='on',top='on',labelbottom='off',labeltop='on',labelsize=8,pad=0)

In [42]:
#V2
#Has Vertical Scale
#Default is 12
mpl.rcParams.update({'font.size': 8})
xin = 3.1
yin = 3.1
img = xin*0.9
y_len = int(img/yin*1000)
colortype = ch.yellowredblue

f = plt.figure(figsize=[3.1,3.3])
ax = f.add_subplot(gs[100:y_len,100:])
im = ax.pcolormesh(X,Y,Z,vmin=0,vmax=1,cmap=colortype)
ax.get_xaxis().get_major_formatter().set_useOffset(False)
plt.xticks([5.579,5.583])
plt.yticks([-30,-20,-10])
ax.set_ylim(xmin,xmax)
ax.set_xlim(bval[0],bval[-1])
plt.ylabel('V$_{p}$ (mV)',size = 12)
plt.xlabel('B (T)',size = 12)
ax.tick_params(axis='y',pad = 2)

cax = f.add_subplot(gs[55:80,400:700])
cbar = plt.colorbar(im,cax=cax,ticks=[0.0,1.0],orientation='horizontal')
cax.xaxis.set_label_position('top')
cax.set_xlabel('R$_D$ (k$\Omega$)',size=9)
cax.set_xticklabels([10.8,11.5])
cax.xaxis.labelpad = 1
cax.tick_params(axis='x',which='both',bottom='on',top='on',labelbottom='off',labeltop='on',labelsize=7,pad=1)

In [43]:
f.savefig('Color Map v2',dpi=1200)

In [169]:
colortype = ch.yellowredblue
f, ax = plt.subplots()
plt.pcolormesh(X,Y,Z,vmin=0,vmax=1,cmap=colortype)
plt.ylim(xmin,xmax)
plt.xlim(bval[0],bval[-1])
plt.ylabel('V$_{p}$ (mV)')
plt.xlabel('B (T)')
cbar = plt.colorbar(shrink=0.5,ticks=[0,1])
cbar.set_label('R$_D$ (k$\Omega$)')
cbar.set_ticklabels([10.8,11.5])
ax.get_xaxis().get_major_formatter().set_useOffset(False)
plt.xticks([5.578,5.581,5.584])
plt.yticks([-30,-20,-10])
plt.show()

In [159]:
f.savefig('Vertical Scale UnNormalized Larger Labels',dpi=1200)

In [168]:
#Default is 12
mpl.rcParams.update({'font.size': 10})

In [17]:
ch.parsePrune('soft')

Use 2011 data:

In [252]:
ch.parsePrune('soft')

In [291]:
bfieldval = bval2011
n = 1024
smoothval = 51
poly = 2
interpdim = 2
data = ch.getPrune()
cmlabel = 'Normalized $\Delta$R$_D$ (k$\Omega$)'
data = [x for (y,x) in sorted(zip(bfieldval,data), key=lambda pair: pair[0])]
bval = sorted(bfieldval)    
xmin = None
xmax = 0
for i in data:
    if i[0][0] > xmin:
        xmin = i[0][0]
    if i[0][-1] < xmax:
        xmax = i[0][-1]
xmin += 0.5
xmax -= 0.5
xv = np.linspace(xmin,xmax,n)
intv = list()
count = 0
for i in data:
    if count == 21:
        xmin += 1
    #Try flipping filtering and interp
    dataclip = clip(i,xmin,xmax)
    dataclip[1] = SG.savitzky_golay(dataclip[1],smoothval,poly)
    tck = interpolate.splrep(dataclip[0], dataclip[1],k=1)
    y = interpolate.splev(xv,tck)
    intv.append(percentile_normalize(y,1))
    count += 1
xb = np.linspace(bval[0],bval[-1],n)
b = list()
for i in range(len(intv[0])):
    hold = list()
    for j in range(len(intv)):
        hold.append(intv[j][i])
    b.append(hold)
intb = list()
for i in b:
    tck = interpolate.splrep(bval,i,k=interpdim)
    y = interpolate.splev(xb,tck)
    intb.append(y)
Z = np.array(intb)
X,Y = pylab.meshgrid(xb,xv)


In [293]:
colortype = ch.yellowredblue
f, ax = plt.subplots()
plt.pcolormesh(X,Y,Z,vmin=0,vmax=1,cmap=colortype)
plt.ylim(xmin,xmax)
plt.xlim(bval[0],bval[-1])
plt.ylabel('Gate Voltage (mV)')
plt.xlabel('B Field (T)')
cbar = plt.colorbar()
cbar.set_label(cmlabel)
ax.get_xaxis().get_major_formatter().set_useOffset(False)
plt.savefig('ColorMap52_2011',dpi=1200)

In [296]:
plt.scatter((se2011[1][:,1]),se2011[1][:,0],s=50,color='w',edgecolor='k')
plt.scatter((se2011[0][:,1]),se2011[0][:,0],s=50,color='#00FFFF',marker='d',edgecolor='k')

7/3 data

In [300]:
ch.parsePrune('soft')
ch.parseData()

In [392]:
bfieldval = bval73
n = 2048
smoothval = 311
poly = 1
interpdim = 2
data = ch.getPrune()
cmlabel = 'Normalized $\Delta$R$_D$ (k$\Omega$)'
data = [x for (y,x) in sorted(zip(bfieldval,data), key=lambda pair: pair[0])]
bval = sorted(bfieldval)    
xmin = None
xmax = 0
for i in data:
    if i[0][0] > xmin:
        xmin = i[0][0]
    if i[0][-1] < xmax:
        xmax = i[0][-1]
xmin += 0.5
xmax -= 0.5
xv = np.linspace(xmin,xmax,n)
intv = list()
count = 0
for i in data:
    if count == 21:
        xmin += 1
    #Try flipping filtering and interp
    dataclip = clip(i,xmin,xmax)
    dataclip[1] = SG.savitzky_golay(dataclip[1],smoothval,poly)
    tck = interpolate.splrep(dataclip[0], dataclip[1],k=1)
    y = interpolate.splev(xv,tck)
    intv.append(percentile_normalize(y,0.25))
    count += 1
xb = np.linspace(bval[0],bval[-1],n)
b = list()
for i in range(len(intv[0])):
    hold = list()
    for j in range(len(intv)):
        hold.append(intv[j][i])
    b.append(hold)
intb = list()
for i in b:
    tck = interpolate.splrep(bval,i,k=interpdim)
    y = interpolate.splev(xb,tck)
    intb.append(y)
Z = np.array(intb)
X,Y = pylab.meshgrid(xb,xv)


In [393]:
colortype = ch.yellowredblue
f, ax = plt.subplots()
plt.pcolormesh(X,Y,Z,vmin=0,vmax=1,cmap=colortype)
plt.ylim(xmin,xmax)
plt.xlim(bval[0],bval[-1])
plt.ylabel('Gate Voltage (mV)')
plt.xlabel('B Field (T)')
cbar = plt.colorbar()
cbar.set_label(cmlabel)
ax.get_xaxis().get_major_formatter().set_useOffset(False)


In [305]:
se73 = np.load('73se.npy')

In [361]:
x = []
y = []
for i in range(len(se73)):
    if len(se73[i])==0:
        continue
    x.extend(se73[i][:,1])
    y.extend(se73[i][:,0])

In [363]:
plt.scatter(x,y,s=50,color='w',edgecolor='k')

In [367]:
ch.interp2D(bval73)
ax.get_xaxis().get_major_formatter().set_useOffset(False)

In [404]:
n=2048
xv = np.linspace(xmin,xmax,n)
for i in data:
    if count == 21:
        xmin += 1
    #Try flipping filtering and interp
    plt.figure()
    dataclip = clip(i,xmin,xmax)
    plt.plot(dataclip[0],dataclip[1])
    dataclip[1] = SG.savitzky_golay(dataclip[1],51,1)
    plt.plot(dataclip[0],dataclip[1],c='r')
    tck = interpolate.splrep(dataclip[0], dataclip[1],k=1)
    y = interpolate.splev(xv,tck)
    plt.plot(xv,y)
    plt.ylim(12,12.4)
    count += 1
    plt.plot()

In [403]:
for i in intv:
    plt.figure()
    plt.plot(xv,i)
    plt.ylim(0,1)

In [396]:
hist73 = np.load('73hist.npy')